In [1]:
# libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

## 인덱스 객체 조사
* 대학교 데이터셋

In [2]:
root = 'C:/Users/wonca/Documents/data/pandas_cookbook/'
file_name = 'college.csv'
college = pd.read_csv(root + file_name)

In [3]:
columns = college.columns

In [4]:
# 인덱스는 변경할 수 없음: Index does not support mutable operations
columns[1] = 'A'

TypeError: Index does not support mutable operations

In [9]:
# 합집합, 교집합, 차집합, 대칭차집합 연산 가능
c1 = columns[:4]
c1

Index(['INSTNM', 'CITY', 'STABBR', 'HBCU'], dtype='object')

In [10]:
c2 = columns[2:6]
c2

Index(['STABBR', 'HBCU', 'MENONLY', 'WOMENONLY'], dtype='object')

In [11]:
c1.union(c2)

Index(['CITY', 'HBCU', 'INSTNM', 'MENONLY', 'STABBR', 'WOMENONLY'], dtype='object')

In [12]:
c1.symmetric_difference(c2)

Index(['CITY', 'INSTNM', 'MENONLY', 'WOMENONLY'], dtype='object')

## 카디션 곱 생성
* 인덱스가 정렬되지않은 데이터프레임간의 연산은 카티션곱으로 수행 => nan 행 발생

In [13]:
s1 = pd.Series(index=list('aac'), data=np.arange(3))
s2 = pd.Series(index=list('acb'), data=np.arange(3))

s1+s2

a    0.0
a    1.0
b    NaN
c    3.0
dtype: float64

## 인덱스 폭발

In [14]:
# 순서가 다른 Series 를 더할 경우 인덱스 개수가 폭발적으로 증가한다.
file_name = 'employee.csv'
employee = pd.read_csv(root + file_name, index_col = 'RACE')
employee.head(2)

UNIQUE_ID               POSITION_TITLE  \
RACE                                                      
Hispanic/Latino          0  ASSISTANT DIRECTOR (EX LVL)   
Hispanic/Latino          1            LIBRARY ASSISTANT   

                                  DEPARTMENT  BASE_SALARY EMPLOYMENT_TYPE  \
RACE                                                                        
Hispanic/Latino  Municipal Courts Department     121862.0       Full Time   
Hispanic/Latino                      Library      26125.0       Full Time   

                 GENDER EMPLOYMENT_STATUS   HIRE_DATE    JOB_DATE  
RACE                                                               
Hispanic/Latino  Female            Active  2006-06-12  2012-10-13  
Hispanic/Latino  Female            Active  2000-07-19  2010-09-18

In [15]:
# BASE_SALAY 선택
salary1 = employee['BASE_SALARY']
salary2 = employee['BASE_SALARY']
salary1 is salary2  # reference 비교

True

In [17]:
# copy로 메모리공간 할당
salary1 = employee['BASE_SALARY'].copy()
salary2 = employee['BASE_SALARY'].copy()
salary1 is salary2  # 메모리 할당

False

In [18]:
# 시리즈 하나만 인덱스 정렬
salary1 = salary1.sort_index()
salary1.head()

RACE
American Indian or Alaskan Native    78355.0
American Indian or Alaskan Native    26125.0
American Indian or Alaskan Native    98536.0
American Indian or Alaskan Native        NaN
American Indian or Alaskan Native    55461.0
Name: BASE_SALARY, dtype: float64

In [19]:
salary2.head()

RACE
Hispanic/Latino    121862.0
Hispanic/Latino     26125.0
White               45279.0
White               63166.0
White               56347.0
Name: BASE_SALARY, dtype: float64

In [22]:
salary_add = salary1 + salary2
print(f'''
salary1 : {len(salary1)}
salary2 : {len(salary2)}
salary_add : {len(salary_add)}
''')


salary1 : 2000
salary2 : 2000
salary_add : 1175424




In [27]:
# 개수 확인
salary_idx_vc = salary1.index.value_counts(dropna=False)
salary_idx_vc.pow(2).sum()

1175424

In [28]:
## 카테시안곱: 모든 경우의 수

## 서로 다른 인데스에 값 채우기

In [29]:
baseball14 = pd.read_csv(root+'baseball14.csv', index_col='playerID')
baseball15 = pd.read_csv(root+'baseball15.csv', index_col='playerID')
baseball16 = pd.read_csv(root+'baseball16.csv', index_col='playerID')


In [31]:
# 14에는 있고, 15에는 없는 인덱스
# 15에는 있고, 14에는 없는 인덱스

baseball14.index.difference(baseball15.index)

Index(['corpoca01', 'dominma01', 'fowlede01', 'grossro01', 'guzmaje01',
       'hoeslj01', 'krausma01', 'preslal01', 'singljo02'],
      dtype='object', name='playerID')

In [32]:
baseball15.index.difference(baseball14.index)

Index(['congeha01', 'correca01', 'gattiev01', 'gomezca01', 'lowrije01',
       'rasmuco01', 'tuckepr01', 'valbulu01'],
      dtype='object', name='playerID')

In [33]:
# 3년간 각 선수의 총 안타 수 

h14 = baseball14['H']
h15 = baseball15['H']
h16 = baseball16['H']

h14.head()

playerID
altuvjo01    225
cartech02    115
castrja01    103
corpoca01     40
dominma01    121
Name: H, dtype: int64

In [34]:
(h14 + h15).head()

playerID
altuvjo01    425.0
cartech02    193.0
castrja01    174.0
congeha01      NaN
corpoca01      NaN
Name: H, dtype: float64

In [41]:
# corpoca01 누락
h_tot = h14.add(h15, fill_value=0).add(h16, fill_value=0)   # 누락값에 0을 더한다.
h_tot.hasnans  # 누락값 검사

False

In [44]:
# 결측값 위치 검사
(baseball14 + baseball15).head(10).style.highlight_null('yellow')  # 양쪽에 결측치가 있을경우

## 다른 DataFrame의 열 추가

In [49]:
## 각부서의 최대연봉을 기록한 새로운 열 추가
employee = pd.read_csv(root + 'employee.csv')
dept_sal = employee[['DEPARTMENT', 'BASE_SALARY']]
dept_sal.head()

DEPARTMENT  BASE_SALARY
0    Municipal Courts Department     121862.0
1                        Library      26125.0
2  Houston Police Department-HPD      45279.0
3  Houston Fire Department (HFD)      63166.0
4    General Services Department      56347.0

In [53]:
dept_sal = dept_sal.sort_values(['DEPARTMENT', 'BASE_SALARY'], ascending=[True, False])

In [58]:
# drop duplicates: department를 기준으로 첫 열만 남긴다.
max_dept_sal = dept_sal.drop_duplicates(subset='DEPARTMENT')
max_dept_sal.head()

# index를 department로 설정
max_dept_sal = max_dept_sal.set_index('DEPARTMENT')
employee = employee.set_index('DEPARTMENT')

In [60]:
# 인덱스가 정렬돼 추가할 수 있다.
# 인덱스가 중복되면 안됨.
employee['MAX_DEPT_SAL'] = max_dept_sal

In [63]:
# 최댓값인지 검사
employee.query('BASE_SALARY > MAX_DEPT_SAL')

Empty DataFrame
Columns: [UNIQUE_ID, POSITION_TITLE, BASE_SALARY, RACE, EMPLOYMENT_TYPE, GENDER, EMPLOYMENT_STATUS, HIRE_DATE, JOB_DATE, MAX_DEPT_SAL]
Index: []

In [64]:
# 인덱스가 중복됐을 경우
np.random.seed(42)
ran_sal = dept_sal.sample(n=10).set_index('DEPARTMENT')
ran_sal

BASE_SALARY
DEPARTMENT                                 
Public Works & Engineering-PWE      34861.0
Houston Airport System (HAS)        29286.0
Houston Police Department-HPD       31907.0
Houston Police Department-HPD       66614.0
Houston Police Department-HPD       42000.0
Houston Police Department-HPD       43443.0
Houston Police Department-HPD       66614.0
Public Works & Engineering-PWE      52582.0
Finance                             93168.0
Houston Police Department-HPD       35318.0

In [65]:
employee['no'] = ran_sal

ValueError: cannot reindex from a duplicate axis

In [67]:
# 인덱스가 부족할 경우 Nan 처리
employee['MAX2'] = max_dept_sal['BASE_SALARY'].head(3)
employee['MAX2'].head()

DEPARTMENT
Municipal Courts Department     NaN
Library                         NaN
Houston Police Department-HPD   NaN
Houston Fire Department (HFD)   NaN
General Services Department     NaN
Name: MAX2, dtype: float64

## 각 열로부터 최댓값 하이라이트

In [68]:
college = pd.read_csv(root+'college.csv', index_col='INSTNM')
college.dtypes

CITY                   object
STABBR                 object
HBCU                  float64
MENONLY               float64
WOMENONLY             float64
RELAFFIL                int64
SATVRMID              float64
SATMTMID              float64
DISTANCEONLY          float64
UGDS                  float64
UGDS_WHITE            float64
UGDS_BLACK            float64
UGDS_HISP             float64
UGDS_ASIAN            float64
UGDS_AIAN             float64
UGDS_NHPI             float64
UGDS_2MOR             float64
UGDS_NRA              float64
UGDS_UNKN             float64
PPTUG_EF              float64
CURROPER                int64
PCTPELL               float64
PCTFLOAN              float64
UG25ABV               float64
MD_EARN_WNE_P10        object
GRAD_DEBT_MDN_SUPP     object
dtype: object

In [69]:
college['MD_EARN_WNE_P10'].iloc[0]  # string

'30300'

In [78]:
# 문자열찾기
college['MD_EARN_WNE_P10'].sort_values(ascending=False).head()

INSTNM
Medical College of Wisconsin                            233100.0
West Virginia School of Osteopathic Medicine            219900.0
A T Still University of Health Sciences                 219800.0
Albany Medical College                                  214400.0
University of Massachusetts Medical School Worcester    213600.0
Name: MD_EARN_WNE_P10, dtype: float64

In [79]:
# to_numeric: 수치형변환
cols = ['MD_EARN_WNE_P10', 'GRAD_DEBT_MDN_SUPP']
for col in cols:
    college[col] = pd.to_numeric(college[col], errors='coerce')
college.dtypes.loc[cols]

MD_EARN_WNE_P10       float64
GRAD_DEBT_MDN_SUPP    float64
dtype: object

In [80]:
college['MD_EARN_WNE_P10'].sort_values(ascending=False)

INSTNM
Medical College of Wisconsin                              233100.0
West Virginia School of Osteopathic Medicine              219900.0
A T Still University of Health Sciences                   219800.0
Albany Medical College                                    214400.0
University of Massachusetts Medical School Worcester      213600.0
                                                            ...   
SAE Institute of Technology  San Francisco                     NaN
Rasmussen College - Overland Park                              NaN
National Personal Training Institute of Cleveland              NaN
Bay Area Medical Academy - San Jose Satellite Location         NaN
Excel Learning Center-San Antonio South                        NaN
Name: MD_EARN_WNE_P10, Length: 7535, dtype: float64

In [85]:
# 최댓값이 정의될 수 없는 열 제외
college_n = college.select_dtypes(include=[np.number])
college_n.head(2)

HBCU  MENONLY  WOMENONLY  RELAFFIL  \
INSTNM                                                                    
Alabama A & M University              1.0      0.0        0.0         0   
University of Alabama at Birmingham   0.0      0.0        0.0         0   

                                     SATVRMID  SATMTMID  DISTANCEONLY  \
INSTNM                                                                  
Alabama A & M University                424.0     420.0           0.0   
University of Alabama at Birmingham     570.0     565.0           0.0   

                                        UGDS  UGDS_WHITE  UGDS_BLACK  ...  \
INSTNM                                                                ...   
Alabama A & M University              4206.0      0.0333      0.9353  ...   
University of Alabama at Birmingham  11383.0      0.5922      0.2600  ...   

                                     UGDS_2MOR  UGDS_NRA  UGDS_UNKN  PPTUG_EF  \
INSTNM                                                                          
Alabama A & M University                0.0000    0.0059     0.0138    0.0656   
University of Alabama at Birmingham     0.0368    0.0179     0.0100    0.2607   

                                     CURROPER  PCTPELL  PCTFLOAN  UG25ABV  \
INSTNM                                                                      
Alabama A & M University                    1   0.7356    0.8284   0.1049   
University of Alabama at Birmingham         1   0.3460    0.5214   0.2422   

                                     MD_EARN_WNE_P10  GRAD_DEBT_MDN_SUPP  
INSTNM                                                                    
Alabama A & M University                     30300.0             33888.0  
University of Alabama at Birmingham          39700.0             21941.5  

[2 rows x 24 columns]

In [87]:
# 이진 컬럼 추출
criteria = college_n.nunique() == 2
criteria.head()

HBCU          True
MENONLY       True
WOMENONLY     True
RELAFFIL      True
SATVRMID     False
dtype: bool

In [89]:
binary_cols = college_n.columns[criteria].tolist()
binary_cols

['HBCU', 'MENONLY', 'WOMENONLY', 'RELAFFIL', 'DISTANCEONLY', 'CURROPER']

In [90]:
## 이진 컬럼 제거
college_n2 = college_n.drop(labels=binary_cols, axis='columns')
college_n2.head()

SATVRMID  SATMTMID     UGDS  UGDS_WHITE  \
INSTNM                                                                         
Alabama A & M University                424.0     420.0   4206.0      0.0333   
University of Alabama at Birmingham     570.0     565.0  11383.0      0.5922   
Amridge University                        NaN       NaN    291.0      0.2990   
University of Alabama in Huntsville     595.0     590.0   5451.0      0.6988   
Alabama State University                425.0     430.0   4811.0      0.0158   

                                     UGDS_BLACK  UGDS_HISP  UGDS_ASIAN  \
INSTNM                                                                   
Alabama A & M University                 0.9353     0.0055      0.0019   
University of Alabama at Birmingham      0.2600     0.0283      0.0518   
Amridge University                       0.4192     0.0069      0.0034   
University of Alabama in Huntsville      0.1255     0.0382      0.0376   
Alabama State University                 0.9208     0.0121      0.0019   

                                     UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  \
INSTNM                                                                 
Alabama A & M University                0.0024     0.0019     0.0000   
University of Alabama at Birmingham     0.0022     0.0007     0.0368   
Amridge University                      0.0000     0.0000     0.0000   
University of Alabama in Huntsville     0.0143     0.0002     0.0172   
Alabama State University                0.0010     0.0006     0.0098   

                                     UGDS_NRA  UGDS_UNKN  PPTUG_EF  PCTPELL  \
INSTNM                                                                        
Alabama A & M University               0.0059     0.0138    0.0656   0.7356   
University of Alabama at Birmingham    0.0179     0.0100    0.2607   0.3460   
Amridge University                     0.0000     0.2715    0.4536   0.6801   
University of Alabama in Huntsville    0.0332     0.0350    0.2146   0.3072   
Alabama State University               0.0243     0.0137    0.0892   0.7347   

                                     PCTFLOAN  UG25ABV  MD_EARN_WNE_P10  \
INSTNM                                                                    
Alabama A & M University               0.8284   0.1049          30300.0   
University of Alabama at Birmingham    0.5214   0.2422          39700.0   
Amridge University                     0.7795   0.8540          40100.0   
University of Alabama in Huntsville    0.4596   0.2640          45500.0   
Alabama State University               0.7554   0.1270          26600.0   

                                     GRAD_DEBT_MDN_SUPP  
INSTNM                                                   
Alabama A & M University                        33888.0  
University of Alabama at Birmingham             21941.5  
Amridge University                              23370.0  
University of Alabama in Huntsville             24097.0  
Alabama State University                        33118.5

In [91]:
max_cols = college_n2.idxmax()  # 컬럼: 최댓값 인덱스
max_cols

SATVRMID                             California Institute of Technology
SATMTMID                             California Institute of Technology
UGDS                                      University of Phoenix-Arizona
UGDS_WHITE                       Mr Leon's School of Hair Design-Moscow
UGDS_BLACK                           Velvatex College of Beauty Culture
UGDS_HISP                       Thunderbird School of Global Management
UGDS_ASIAN                          Cosmopolitan Beauty and Tech School
UGDS_AIAN                             Haskell Indian Nations University
UGDS_NHPI                                       Palau Community College
UGDS_2MOR                                                 LIU Brentwood
UGDS_NRA               California University of Management and Sciences
UGDS_UNKN             Le Cordon Bleu College of Culinary Arts-San Fr...
PPTUG_EF                        Thunderbird School of Global Management
PCTPELL                                        MTI Business Coll

In [92]:
unique_max_cols = max_cols.unique()
unique_max_cols[:5]

array(['California Institute of Technology',
       'University of Phoenix-Arizona',
       "Mr Leon's School of Hair Design-Moscow",
       'Velvatex College of Beauty Culture',
       'Thunderbird School of Global Management'], dtype=object)

In [98]:
# 하이라이트
college_n2.loc[unique_max_cols].style.highlight_max(color='gray')  # sty

In [100]:
# 각 행의 최댓값
college = pd.read_csv(root+'college.csv', index_col='INSTNM')
college_udgs = college.filter(like='UGDS_').head()
college_udgs.style.highlight_max(axis='columns')

## 메서드 체인을 활용한 idxmax 복제

In [101]:
# 최대열값을 가진 인덱스
# 수치열만 뽑기
cols = ['MD_EARN_WNE_P10', 'GRAD_DEBT_MDN_SUPP']
for col in cols:
    college[col] = pd.to_numeric(college[col], errors='coerce')
college_n = college.select_dtypes(include=[np.number])
criteria = college_n.nunique() == 2
bianary_cols = college_n.columns[criteria].tolist()
college_n = college_n.drop(bianary_cols, axis=1)

In [104]:
# 각 열의 최댓값
college_n.max()

SATVRMID                 765.0000
SATMTMID                 785.0000
UGDS                  151558.0000
UGDS_WHITE                 1.0000
UGDS_BLACK                 1.0000
UGDS_HISP                  1.0000
UGDS_ASIAN                 0.9727
UGDS_AIAN                  1.0000
UGDS_NHPI                  0.9983
UGDS_2MOR                  0.5333
UGDS_NRA                   0.9286
UGDS_UNKN                  0.9027
PPTUG_EF                   1.0000
PCTPELL                    1.0000
PCTFLOAN                   1.0000
UG25ABV                    1.0000
MD_EARN_WNE_P10       233100.0000
GRAD_DEBT_MDN_SUPP     49750.0000
dtype: float64

In [105]:
college_n.eq(college_n.max()).head()

SATVRMID  SATMTMID   UGDS  UGDS_WHITE  \
INSTNM                                                                       
Alabama A & M University                False     False  False       False   
University of Alabama at Birmingham     False     False  False       False   
Amridge University                      False     False  False       False   
University of Alabama in Huntsville     False     False  False       False   
Alabama State University                False     False  False       False   

                                     UGDS_BLACK  UGDS_HISP  UGDS_ASIAN  \
INSTNM                                                                   
Alabama A & M University                  False      False       False   
University of Alabama at Birmingham       False      False       False   
Amridge University                        False      False       False   
University of Alabama in Huntsville       False      False       False   
Alabama State University                  False      False       False   

                                     UGDS_AIAN  UGDS_NHPI  UGDS_2MOR  \
INSTNM                                                                 
Alabama A & M University                 False      False      False   
University of Alabama at Birmingham      False      False      False   
Amridge University                       False      False      False   
University of Alabama in Huntsville      False      False      False   
Alabama State University                 False      False      False   

                                     UGDS_NRA  UGDS_UNKN  PPTUG_EF  PCTPELL  \
INSTNM                                                                        
Alabama A & M University                False      False     False    False   
University of Alabama at Birmingham     False      False     False    False   
Amridge University                      False      False     False    False   
University of Alabama in Huntsville     False      False     False    False   
Alabama State University                False      False     False    False   

                                     PCTFLOAN  UG25ABV  MD_EARN_WNE_P10  \
INSTNM                                                                    
Alabama A & M University                False    False            False   
University of Alabama at Birmingham     False    False            False   
Amridge University                      False    False            False   
University of Alabama in Huntsville     False    False            False   
Alabama State University                False    False            False   

                                     GRAD_DEBT_MDN_SUPP  
INSTNM                                                   
Alabama A & M University                          False  
University of Alabama at Birmingham               False  
Amridge University                                False  
University of Alabama in Huntsville               False  
Alabama State University                          False

In [107]:
college_n.eq(college_n.max()).cumsum().cumsum()  # 1이 최초출현하는 지점이 최댓값

SATVRMID  SATMTMID  UGDS  \
INSTNM                                                                         
Alabama A & M University                                   0         0     0   
University of Alabama at Birmingham                        0         0     0   
Amridge University                                         0         0     0   
University of Alabama in Huntsville                        0         0     0   
Alabama State University                                   0         0     0   
...                                                      ...       ...   ...   
SAE Institute of Technology  San Francisco              7305      7305   415   
Rasmussen College - Overland Park                       7306      7306   416   
National Personal Training Institute of Cleveland       7307      7307   417   
Bay Area Medical Academy - San Jose Satellite L...      7308      7308   418   
Excel Learning Center-San Antonio South                 7309      7309   419   

                                                    UGDS_WHITE  UGDS_BLACK  \
INSTNM                                                                       
Alabama A & M University                                     0           0   
University of Alabama at Birmingham                          0           0   
Amridge University                                           0           0   
University of Alabama in Huntsville                          0           0   
Alabama State University                                     0           0   
...                                                        ...         ...   
SAE Institute of Technology  San Francisco              379750       73107   
Rasmussen College - Overland Park                       379859       73135   
National Personal Training Institute of Cleveland       379968       73163   
Bay Area Medical Academy - San Jose Satellite L...      380077       73191   
Excel Learning Center-San Antonio South                 380186       73219   

                                                    UGDS_HISP  UGDS_ASIAN  \
INSTNM                                                                      
Alabama A & M University                                    0           0   
University of Alabama at Birmingham                         0           0   
Amridge University                                          0           0   
University of Alabama in Huntsville                         0           0   
Alabama State University                                    0           0   
...                                                       ...         ...   
SAE Institute of Technology  San Francisco             341103         983   
Rasmussen College - Overland Park                      341239         984   
National Personal Training Institute of Cleveland      341375         985   
Bay Area Medical Academy - San Jose Satellite L...     341511         986   
Excel Learning Center-San Antonio South                341647         987   

                                                    UGDS_AIAN  UGDS_NHPI  \
INSTNM                                                                     
Alabama A & M University                                    0          0   
University of Alabama at Birmingham                         0          0   
Amridge University                                          0          0   
University of Alabama in Huntsville                         0          0   
Alabama State University                                    0          0   
...                                                       ...        ...   
SAE Institute of Technology  San Francisco              11382       3316   
Rasmussen College - Overland Park                       11384       3317   
National Personal Training Institute of Cleveland       11386       3318   
Bay Area Medical Academy - San Jose Satellite L...      11388       3319   
Excel Learning Center-San Antonio South                 11390       3320   

     

In [109]:
has_row_max2 = college_n.eq(college_n.max()).cumsum().cumsum().eq(1).any(axis='columns')
has_row_max2

INSTNM
Alabama A & M University                                  False
University of Alabama at Birmingham                       False
Amridge University                                        False
University of Alabama in Huntsville                       False
Alabama State University                                  False
                                                          ...  
SAE Institute of Technology  San Francisco                False
Rasmussen College - Overland Park                         False
National Personal Training Institute of Cleveland         False
Bay Area Medical Academy - San Jose Satellite Location    False
Excel Learning Center-San Antonio South                   False
Length: 7535, dtype: bool

In [110]:
idxmax_cols = has_row_max2[has_row_max2].index
idxmax_cols ## 느림

Index(['Thunderbird School of Global Management',
       'Southwest University of Visual Arts-Tucson', 'ABC Beauty College Inc',
       'Velvatex College of Beauty Culture',
       'California Institute of Technology',
       'Le Cordon Bleu College of Culinary Arts-San Francisco',
       'MTI Business College Inc', 'Dongguk University-Los Angeles',
       'Mr Leon's School of Hair Design-Moscow',
       'Haskell Indian Nations University', 'LIU Brentwood',
       'Medical College of Wisconsin', 'Palau Community College',
       'California University of Management and Sciences',
       'Cosmopolitan Beauty and Tech School', 'University of Phoenix-Arizona'],
      dtype='object', name='INSTNM')

## 가장 보편적인 최댓값 검색

In [112]:
college_udgs = college.filter(like='UGDS_')
college_udgs.head()

UGDS_WHITE  UGDS_BLACK  UGDS_HISP  \
INSTNM                                                                   
Alabama A & M University                 0.0333      0.9353     0.0055   
University of Alabama at Birmingham      0.5922      0.2600     0.0283   
Amridge University                       0.2990      0.4192     0.0069   
University of Alabama in Huntsville      0.6988      0.1255     0.0382   
Alabama State University                 0.0158      0.9208     0.0121   

                                     UGDS_ASIAN  UGDS_AIAN  UGDS_NHPI  \
INSTNM                                                                  
Alabama A & M University                 0.0019     0.0024     0.0019   
University of Alabama at Birmingham      0.0518     0.0022     0.0007   
Amridge University                       0.0034     0.0000     0.0000   
University of Alabama in Huntsville      0.0376     0.0143     0.0002   
Alabama State University                 0.0019     0.0010     0.0006   

                                     UGDS_2MOR  UGDS_NRA  UGDS_UNKN  
INSTNM                                                               
Alabama A & M University                0.0000    0.0059     0.0138  
University of Alabama at Birmingham     0.0368    0.0179     0.0100  
Amridge University                      0.0000    0.0000     0.2715  
University of Alabama in Huntsville     0.0172    0.0332     0.0350  
Alabama State University                0.0098    0.0243     0.0137

In [115]:
h_percentage_race = college_udgs.idxmax(axis=1)
h_percentage_race.value_counts(normalize=True)

UGDS_WHITE    0.670352
UGDS_BLACK    0.151586
UGDS_HISP     0.129473
UGDS_UNKN     0.023422
UGDS_ASIAN    0.012074
UGDS_AIAN     0.006110
UGDS_NRA      0.004073
UGDS_NHPI     0.001746
UGDS_2MOR     0.001164
dtype: float64

In [122]:
# 흑인비율이 가장 많은 학교 중 두번쨰로 많은 인종의 비율은?

college_black = college_udgs[h_percentage_race == 'UGDS_BLACK']
college_black = college_black.drop('UGDS_BLACK', axis =1)  # 흑인비율 제거
# 다시 최댓값
college_black.idxmax(axis=1).value_counts(normalize=True)

UGDS_WHITE    0.661228
UGDS_HISP     0.230326
UGDS_UNKN     0.071977
UGDS_NRA      0.018234
UGDS_ASIAN    0.009597
UGDS_2MOR     0.006718
UGDS_NHPI     0.000960
UGDS_AIAN     0.000960
dtype: float64